In [ ]:
## Simple GenAI App Using Langchain


In [ ]:
# install ipykernel which is required
!pip install ipykernel

In [ ]:
# To setup and loading all the environment variables
import os 
from dotenv import load_dotenv
load_dotenv.load()

In [ ]:
# Setting up OpenAI and Lanchain API Keys in os environment

os.environ["OPENAI_API_KEY"] = os.getenv["OPENAI_API_KEY"]

# langchain api key -> specifically for tracking using langsmith as it is a component of langchain
os.environ["LANGCHAIN_API_KEY"] = os.getenv["LANGCHAIN_API_KEY"]

# for tracking 
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv["LANGCHAIN_PROJECT"]

In [ ]:
# Data Ingestion -> from the website we need to scrape the data
# WebBaseLoader -> scrape the data from website
# WebBaseLoader -> internally use bs4
from langchain_community.document_loaders import WebBaseLoader

In [ ]:
loader = WebBaseLoader("https://docs.smith.langchain.com/administration/tutorials/manage_spend")
loader

In [ ]:
docs = loader.load()
docs

In [ ]:
# Load Data -> Docs -> Divide document into chunk documents -> vectors (vectors embedding) -> vectorstore db

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunksize=1000 chunk_overlap=200)
documents = text_splitter.split_documents(docs)
documents

In [ ]:
# convert into vectors using openaiembeddings
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [ ]:
from langchain_community import FAISS
vectorstoredb = FAISS.from_documents(documents,embeddings)

In [ ]:
vectorstoredb

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model ="gpt-4o")

In [ ]:
# Query from a vectorstoredb
query = "Langsmith has two usage limits: total traces and extended"
result = vectorstoredb.similarity_search(query)
result[0].page_content

In [ ]:
# Retrieval Chain, Document Chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


prompt = ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>"""
)


document_chain = create_stuff_documents_chain(llm,prompt)
document_chain

In [ ]:
from langchain_core.documents import Document
document_chain.invoke({
    "input": "Langsmith has two usage limits: total traces and extended",
    "context" :[Document(page_content="Langsmith has two usage limits: total traces and extended traces")]

})

In [ ]:
# Retriever -> pathway to get the information from the vectorstoredb 
# Input -> Retriever -> vectorstoredb
retriever = vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever,document_chain)    # document_chain -> context information

In [ ]:
retrieval_chain

In [ ]:
# Get the response from the llm 
response = retrieval_chain.invoke({"input": "Langsmith has two usage limit: total traces and extendedr"})
response["answer"]

In [ ]:
response

In [ ]:
response['context']